In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import lightgbm as lgb
import plotly.express as px
from itertools import combinations
import os
import gc
import sys

import warnings
warnings.filterwarnings("ignore")

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [3]:
# Load the optiver files

train_df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
revealed_targets_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
test_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv')

In [4]:
# Overview of the optiver data

train_df.sort_values(by=['stock_id','date_id'],inplace=True)
train_df.loc[train_df['stock_id']==0]

train_df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
191,0,0,10,1299772.70,1,1.000026,15261106.63,NaN,NaN,0.999812,13996.50,1.000026,23519.16,0.999892,0.389814,1,0_10_0
382,0,0,20,1299772.70,1,0.999919,15261106.63,NaN,NaN,0.999812,4665.50,0.999919,12131.60,0.999842,4.220009,2,0_20_0
573,0,0,30,1299772.70,1,1.000133,15261106.63,NaN,NaN,1.000026,55998.00,1.000133,46203.30,1.000085,5.450249,3,0_30_0
764,0,0,40,1218204.43,1,1.000455,15342674.90,NaN,NaN,1.000241,14655.95,1.000455,26610.45,1.000317,3.169775,4,0_40_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237179,199,480,500,2661783.46,-1,1.002764,23969215.23,1.000859,1.001494,1.002764,122246.18,1.003082,425296.17,1.002835,-7.209778,26450,480_500_199
5237379,199,480,510,2658917.61,-1,1.002764,23972081.09,1.000859,1.001494,1.002764,677012.04,1.003082,347268.87,1.002974,-9.750128,26451,480_510_199
5237579,199,480,520,1352558.88,-1,1.001812,23978379.68,1.000859,1.001494,1.001812,225361.65,1.002129,194630.52,1.001982,3.629923,26452,480_520_199
5237779,199,480,530,1352558.88,-1,1.001812,23978379.68,1.000859,1.001494,1.001812,285559.05,1.002129,214513.32,1.001993,4.760027,26453,480_530_199


In [5]:
def feature_engineering(df):
    
    #Removing data that's not in the actual test data in the  API
    cols = [c for c in df.columns if c not in ['row_id', 'time_id', 'target']]
    
    df = df[cols]
    
    # Here we calculate spread differentials and spread ratios
    
#     df['bid_minus_ask_size'] = df['bid_size']- df ['ask_size']
    df['bid_ask_size_ratio'] = df['bid_size']/df['ask_size']
    df['bid_minus_ask_spread'] = df['ask_price'] - df['bid_price']
    
    # Next we calculate median, matched sizes, bid and ask sizes,
    # This will allow us to determine if we are trading thinly or otherwise
    # Some of the daily median values might not be super sueful in the actual implementation of the model as the data begins to
    # populate
    
    
    stock_median_ask_size = df.groupby(['stock_id'])['ask_size'].median()
    stock_median_bid_size = df.groupby(['stock_id'])['bid_size'].median()
    stock_median_imbalance_size = df.groupby(['stock_id'])['imbalance_size'].median()
    
    day_median_ask_size = df.groupby(['date_id'])['ask_size'].median()
    day_median_bid_size = df.groupby(['date_id'])['bid_size'].median()
    day_median_imbalance_size = df.groupby(['date_id'])['imbalance_size'].median()
    
    day_stock_ask_median_ask_size = df.groupby(['date_id','stock_id'])['ask_size'].median()
    day_stock_bid_median_ask_size = df.groupby(['date_id','stock_id'])['bid_size'].median()
    day_stock_imbalance_median_ask_size = df.groupby(['date_id','stock_id'])['imbalance_size'].median()
    
    df = df.merge(stock_median_ask_size, how= 'left', left_on='stock_id', right_index=True, suffixes=('','s_stock_median'))
    df = df.merge(stock_median_bid_size, how= 'left', left_on='stock_id', right_index=True, suffixes=('','_stock_median'))
    df = df.merge(stock_median_imbalance_size, how= 'left', left_on='stock_id', right_index=True, suffixes=('','_stock_median'))
    
     # Lag features
#     lag_features = ['bid_size', 'ask_size', 'imbalance_size']
#     for feature in lag_features:
#         df[f'{feature}_lag_1'] = df.groupby(['stock_id'])[feature].shift(1)

    df[f'imbalance_size_lag_1'] = df.groupby(['stock_id'])['imbalance_size'].shift(1)
    
#     # Rolling imbalance_size with a single window size
    
    #window_size = 5
    df[f'imbalance_size_rolling_mean_5'] = df.groupby(['stock_id'])['imbalance_size'].transform(lambda x: x.rolling(window=5).mean())
    df[f'imbalance_size_rolling_mean_15'] = df.groupby(['stock_id'])['imbalance_size'].transform(lambda x: x.rolling(window=15).mean())

    
    df.drop(
        columns=['date_id'],
        axis=1,
        inplace = True
    )
    
    gc.collect()
    
    return df

In [6]:
y = train_df['target'].values
X = feature_engineering(train_df)

In [ ]:
#X.tail(10)
#X.columns

In [7]:
model = lgb.LGBMRegressor(objective='mae', 
                      n_estimators=600, 
                      random_state=51, num_threads=4, num_leaves=35)

In [8]:
model.fit(X, y)

[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4


LGBMRegressor(n_estimators=600, num_leaves=35, num_threads=4, objective='mae',
              random_state=51)

In [9]:
len(model.feature_importances_)

21

In [10]:
feature_imp = pd.Series(model.feature_importances_, index=X.columns).sort_values()
print('Columns with poor contribution', feature_imp[feature_imp<15].index)
fig = px.bar(x=feature_imp, y=feature_imp.index, orientation='h')
fig.show()

Columns with poor contribution Index([], dtype='object')


In [12]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [13]:
counter = 0
Y_actual = pd.DataFrame()
for (test, revealed_targets, sample_prediction) in iter_test:
    x = feature_engineering(test)
    x.drop("currently_scored", axis=1, inplace=True)
    sample_prediction['target'] = model.predict(x)
    env.predict(sample_prediction)
    #concat
    Y_actual = pd.concat( [Y_actual, revealed_targets])
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [15]:
# x.head(10)

In [ ]:
# x = feature_engineering(test)
# print(x.columns)  # Print the feature names before prediction
# sample_prediction['target'] = model.predict(x)

In [ ]:
# len(x.columns)